In [6]:
%load_ext autoreload 
%autoreload 2
from general.GeneralUtil import MinosPythonWrapper

from minos.MinosData import MinosData
import os



The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [ ]:
def process_trial(trial_data, filtered_start='Start', filtered_end='End_Correct'):
    """ Function for processing the trial data into
        a more organized manner.
    """
    processed_trial = dict()
    for i in range(len(trial_data['Number'])):
        trial_num = trial_data['Number'][i]
        # initilization trial
        if trial_num == 0:
            continue
        # valid trial
        if trial_num not in processed_trial:
            processed_trial[trial_num] = dict()
        processed_trial[trial_num][trial_data['Event'][i]] = trial_data['Timestamp'][i]
    
    valid_end_condition = dict()

    for k in list(processed_trial.keys()):
        if not filtered_start in processed_trial[k]:
            del processed_trial[k]
        else:
            if filtered_end !='End' and not filtered_end in processed_trial[k]:
                del processed_trial[k]
            elif filtered_end =='End':
                check_valid = [cur for cur in ['End', 'End_Correct', 'End_Miss', 'End_Wrong'] if cur in processed_trial[k]]
                if len(check_valid) == 0:
                    del processed_trial[k]
                
                for end_cond in check_valid:
                    valid_end_condition[end_cond] = 1

    # fill in all end condition (e.g., fill nan in End_Miss for correct trials)
    for trial_num in processed_trial:
        for end_cond in valid_end_condition:
            if end_cond not in processed_trial[trial_num]:
                processed_trial[trial_num][end_cond] = np.nan

    return processed_trial

def find_closest(number, num_list):
    diff = np.abs(number-num_list)
    return np.argmin(diff)


def align_trial(trial_num, behavior_data, trial_data, start_event='Start', end_event='End_Correct'):
    """ Function that aligns the start and end time of trial data
        from three files.
        
        Inputs:
            trial_num: internal trial id for Minos.
            behavior_data: player/eye data file.
            trial_data: trial data file.
            start_event: event indicating the start of the stage
            end_event: event indicating the end of the stage
        
        Return:
            [start_idx, end_idx]: corresponding start and end indices 
            in the player/eye data for the current trial.
    """
    cur_trial = trial_data[trial_num]
    start_time = cur_trial[start_event]
    if end_event == 'End' and end_event not in cur_trial:
        end_event = [cur for cur in ['End_Correct', 'End_Miss', 'End_Wrong'] 
                    if cur in cur_trial and not np.isnan(cur_trial[cur])][0]    
    end_time = cur_trial[end_event]
    start_idx = find_closest(start_time, behavior_data['Timestamp'])
    end_idx = find_closest(end_time, behavior_data['Timestamp'])

    assert start_idx<=end_idx, "Misaligned trial detected"
    return start_idx, end_idx


In [13]:
from glob import glob
import re
from general.GeneralUtil import process_trial_info
import numpy as np

def MinosPythonWrapper(minos_dir):
    """ Super-script for reading raw Minos data from the output directory, and merging them 
    into one dictionary. Note that the code only process the behavioral data, without synchronization. 
    For synchronization/integration of neural data, please further use MinosIO.

    Input:
        minos_dir: Directory storing raw Minos files.

    Return:
        A dictionary storing the behavioral data for all available paradigms.
    """
    # pre specify a set of keyword related to time
    time_related_key = ['Timestamp', 'Start_Align', 'End', 'End_Miss', 'End_Correct', 'Loading', 'Align_Loading',
                        'On', 'Off', 'End_Wrong', 'ParadigmStop', 'ParadigmStart']

    # load the eye, player, reward and sync data independent of paradigms
    eye_data = MinosData(os.path.join(minos_dir, 'Minos', 'Eye.bin')).Values
    player_data = MinosData(os.path.join(minos_dir, 'Minos', 'Player.bin')).Values
    reward_data = MinosData(os.path.join(minos_dir, 'Minos', 'Reward.bin')).Values
    sync_data = MinosData(os.path.join(minos_dir, 'Minos', 'Sync.bin')).Values
    sync_start = sync_data['Timestamp'][0] # use the first sync time as start time of the system

    # iterate through all paradigms
    processed_trial = dict()
    all_paradigm = [os.path.basename(cur).replace(' ', '') for cur in glob(os.path.join(minos_dir, 'Minos', '*')) 
                if os.path.isdir(cur) and 'Assets' not in cur]
    for paradigm in all_paradigm:
        processed_trial[paradigm] = dict()
        tmp_trial_data = MinosData(os.path.join(minos_dir, 'Minos', ' '.join(re.split(r'(?<!^)(?=[A-Z])', paradigm)), 
                                'Trials.bin')).Values
        tmp_trial_info = MinosData(os.path.join(minos_dir, 'Minos', ' '.join(re.split(r'(?<!^)(?=[A-Z])', paradigm)), 
                                'TrialInfo.bin')).Values
        tmp_trial_info = process_trial_info(tmp_trial_info, paradigm)
        for k in tmp_trial_info:
            processed_trial[paradigm][k] = []

        if paradigm != 'PolyFaceNavigator':
            tmp_trial_data = process_trial(tmp_trial_data, filtered_start='Start_Align', filtered_end='End')
            processed_trial[paradigm]['Eye'] = []
        else:   
            tmp_trial_data = process_trial(tmp_trial_data, filtered_start='Start', filtered_end='End')
            processed_trial[paradigm]['Player'] = [] 
            processed_trial[paradigm]['Eye_cue'] = []
            processed_trial[paradigm]['Eye_arena'] = []
            processed_trial[paradigm]['isContinuous'] = []
            processed_trial[paradigm]['Reward'] = []

        for idx in range(len(tmp_trial_info['Number'])):
            trial_num = tmp_trial_info['Number'][idx]
            if trial_num not in tmp_trial_data:
                continue
            if paradigm != 'PolyFaceNavigator':
                # align eye data
                start_idx, end_idx = align_trial(trial_num, eye_data, tmp_trial_data, 'Start_Align', 'End')
                aligned_eye = {k: eye_data[k][start_idx:end_idx] for k in eye_data}
                processed_trial[paradigm]['Eye'].append(aligned_eye)
            else:        
                # align eye data during cue phase
                start_idx, end_idx = align_trial(trial_num, eye_data, tmp_trial_data, 'On', 'Off')
                aligned_eye_cue = {k: eye_data[k][start_idx:end_idx] for k in eye_data}

                # align eye data during navigation phase
                start_idx, end_idx = align_trial(trial_num, eye_data, tmp_trial_data, 'Start', 'End')
                aligned_eye_arena = {k: eye_data[k][start_idx:end_idx] for k in eye_data}

                # align player data
                start_idx, end_idx = align_trial(trial_num, player_data, tmp_trial_data, 'Start', 'End')
                aligned_player = {k: player_data[k][start_idx:end_idx] for k in player_data}

                # align reward
                start_time, end_time = aligned_eye_arena['Timestamp'][0], aligned_eye_arena['Timestamp'][-1]
                aligned_reward = [cur for cur in reward_data['Timestamp'] if cur >=start_time and cur<=end_time]
                aligned_reward = {'Timestamp': aligned_reward}

                processed_trial[paradigm]['Eye_cue'].append(aligned_eye_cue)
                processed_trial[paradigm]['Eye_arena'].append(aligned_eye_arena)
                processed_trial[paradigm]['Player'].append(aligned_player)
                processed_trial[paradigm]['Reward'].append(aligned_reward)

            # merging data from trial data
            for k in tmp_trial_data[trial_num]:
                if k not in processed_trial[paradigm]:
                    processed_trial[paradigm][k] = []
                processed_trial[paradigm][k].append(tmp_trial_data[trial_num][k])
            
            # merge data from trial info
            for k in tmp_trial_info:
                if k not in processed_trial[paradigm]:
                    processed_trial[paradigm][k] = []
                processed_trial[paradigm][k].append(tmp_trial_info[k][idx])                

    # correct the time based on the first sync time (from 100ns to second)
    for k in processed_trial:
        for t_keyword in time_related_key:
            if t_keyword in processed_trial[k]:
                processed_trial[k][t_keyword] = [(t-sync_start)/1e7 if not np.isnan(t) else t for t in processed_trial[k][t_keyword]]
        for trial_idx in range(len(processed_trial[k]['Number'])):
            if 'Eye' in processed_trial[k]:
                processed_trial[k]['Eye'][trial_idx]['Timestamp'] = [(t-sync_start)/1e7 for t in processed_trial[k]['Eye'][trial_idx]['Timestamp']]
            else:
                processed_trial[k]['Eye_cue'][trial_idx]['Timestamp'] = [(t-sync_start)/1e7 for t in processed_trial[k]['Eye_cue'][trial_idx]['Timestamp']]
                processed_trial[k]['Eye_arena'][trial_idx]['Timestamp'] = [(t-sync_start)/1e7 for t in processed_trial[k]['Eye_arena'][trial_idx]['Timestamp']]
                processed_trial[k]['Player'][trial_idx]['Timestamp'] = [(t-sync_start)/1e7 for t in processed_trial[k]['Player'][trial_idx]['Timestamp']]
                processed_trial[k]['Reward'][trial_idx]['Timestamp'] = [(t-sync_start)/1e7 for t in processed_trial[k]['Reward'][trial_idx]['Timestamp']]

    sync_data['Timestamp'] = [(t-sync_start)/1e7 for t in sync_data['Timestamp']] 

    return {'Paradigm': processed_trial, 'Sync':sync_data}


In [25]:
# load data
minos_dir = '/mnt/y/UserData/seoyoung/MinosData/241222_163214_Debug-Jamie'
data = MinosPythonWrapper(minos_dir)
print(data['Paradigm']['FiveDot'].keys())
print(data['Sync'].keys())

dict_keys(['Timestamp', 'Number', 'Size', 'Pos', 'Eye', 'Start_Align', 'End'])
dict_keys(['Timestamp', 'Number'])


In [ ]:
# visualize data


def overlay_heatmap(img, att, cmap=plt.cm.jet):
    gamma = 1.0
    att = cv2.blur(att, (35, 35))
    colorized = cmap(np.uint8(att*255))
    alpha = 0.7
    overlaid = np.uint8(img*(1-alpha)+colorized[:, :, 2::-1]*255*alpha)
    return overlaid

if args.load_json is None:
    # create the mapping from ID to image name
    stim2id = json.load(open(args.stim2id))
    id2stim = dict()
    for k in stim2id:
        id2stim[stim2id[k]] = k.split('\\')[-1]

    trial_info = MinosData(os.path.join(args.trial_record, 'Passive Fixation', 'TrialInfo.bin')).Values
    trial_data = MinosData(os.path.join(args.trial_record, 'Passive Fixation', 'Trials.bin')).Values
    eye_data = MinosData(os.path.join(args.trial_record, 'Eye.bin')).Values

    # preprocess the data for better structure
    trial_data = process_trial(trial_data, 'Start_Align', 'End')
    trial2stim = get_stimulus_mapping(trial_info, id2stim)

    # iterate through all trial and extract the fixation 
    avg_num_fixation = []
    record_data = dict()
    for trial_num in trial_data:
        # align the eye data 
        start_idx, end_idx = align_trial(trial_num, eye_data, trial_data, 'Start_Align', 'End')
        eye_x = [float(eye_data['ConvergenceX'][idx]) for idx in range(start_idx, end_idx+1)]
        eye_y = [float(eye_data['ConvergenceY'][idx]) for idx in range(start_idx, end_idx+1)]
        eye_z = [float(eye_data['ConvergenceZ'][idx]) for idx in range(start_idx, end_idx+1)]
        eye_x, eye_y = MinosEyeConversion(eye_x, eye_y, eye_z, 50)

        # from 100 ns to ms
        eye_time = [float(eye_data['Timestamp'][idx])/1e+4 for idx in range(start_idx, end_idx+1)]
        tmp_eye = {'x': eye_x, 'y': eye_y, 'time': eye_time}
        fixation = extract_fixations(tmp_eye, 0.3, 0.12, 80) # originally 0.25, 0.1, 150
        record_data[trial2stim[trial_num]] = [[fix[0], fix[1]] for fix in fixation]
        avg_num_fixation.append(len(fixation))

    # basic stat
    print('Average number of fixations %.2f, STD: %.2f' %(np.mean(avg_num_fixation), np.std(avg_num_fixation)))

    os.makedirs(args.save_dir, exist_ok=True)
    with open(os.path.join(args.save_dir, 'fixation.json'), 'w') as f:
        json.dump(record_data, f)
else:
    record_data = json.load(open(args.load_json))

# visualization
if args.vis_map:
    os.makedirs(os.path.join(args.save_dir, 'saliency_map'), exist_ok=True)
    os.makedirs(os.path.join(args.save_dir, 'fixation_map'), exist_ok=True)
    os.makedirs(os.path.join(args.save_dir, 'overlay_map'), exist_ok=True)
    for img in record_data:
        cur_img = cv2.imread(os.path.join(args.img_dir, img))
        cur_fix = record_data[img]
        fix_map = np.zeros([args.img_h, args.img_w])
        for (x, y) in cur_fix:
            if x<1 and x>=0 and y>=0 and y<1:
                y = int(y*args.img_h)
                x = int(x*args.img_w)
                fix_map[y, x] = 1
        
        cv2.imwrite(os.path.join(args.save_dir, 'fixation_map', img), fix_map*255)

        # convert fixation map to saliency maps
        sal_map = gaussian_filter(fix_map, sigma=50)
        sal_map /= sal_map.max()
        cv2.imwrite(os.path.join(args.save_dir, 'saliency_map', img), sal_map*255)

        # overlay map
        overlay_img = overlay_heatmap(cur_img, sal_map)
        cv2.imwrite(os.path.join(args.save_dir, 'overlay_map', img), overlay_img)


